In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import glob

import tifffile as tf

import os

path = "/home/patrick/github/data"

filenames = glob.glob(os.path.join(path, "dm*/**/*.tif*"), recursive=True)

print(len(filenames))

In [ ]:
# filter lamella, landing

print(len(filenames))
# filenames = [fname for fname in filenames if "ref_landing" not in fname and "ref_lamella" not in fname and "trench" not in fname and "low_res" not in fname]

filenames = [fname for fname in filenames if "needle" in fname]
print(len(filenames))

In [ ]:
for fname in filenames[:10]:
    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
save_path = "/home/patrick/github/data/training"
for i, fname in enumerate(filenames):
    
    img = tf.imread(fname)
    new_fname = os.path.join(save_path,f"a{i:05d}.tif")

    tf.imsave(new_fname, img)


In [ ]:
train_filenames = sorted(glob.glob(os.path.join(save_path, "*.tif")))
for fname in train_filenames[:10]:

    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
import zarr

images = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/training/", "images", "*.tif*"), aszarr=True)) 
labels = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/training/", "labels","*.tif*"), aszarr=True))


print(images.shape)
print(labels.shape)


for img, label in zip(images, labels):


    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(label)
    plt.show()



### Model Testing

In [ ]:
%load_ext autoreload
%autoreload 2 


import glob
import os
import tifffile as tf
import matplotlib.pyplot as plt

images = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images", "*.tif*")))
labels = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/labels", "*.tif*")))

for ii, (i, l) in enumerate(zip(images, labels)):
    print(os.path.basename(i), os.path.basename(l))

    img, lbl = tf.imread(i), tf.imread(l)

    fig, ax = plt.subplots(1, 2, figsize=(7, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")
    ax[1].imshow(lbl)
    ax[1].set_title("Label")
    plt.show()
    
    if ii == 2:
        break



In [ ]:
filenames = sorted(glob.glob(os.path.join("/home/patrick/github/data/liftout/training/train/images/", "*.tif*")))

print("files: ", len(filenames))
# filenames.append(*list(sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images/", "*.tif*")))
print("files: ", len(filenames))
import random
random.shuffle(filenames)


from fibsem.segmentation.model import SegmentationModel

baseline = "/home/patrick/github/fibsem/fibsem/segmentation/models/model.pt"
# checkpoint_2 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_01_51_n08_model.pt"
# checkpoint_5 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_00_58_n05_model.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_02_27_n10_model.pt"
# checkpoints = [baseline, checkpoint_2, checkpoint_5, checkpoint_10]

small_model = "/home/patrick/github/fibsem/fibsem/segmentation/models/model2.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_00_10_n10_model.pt"
# checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_09_22_n15_model.pt"
# checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_18_32_n20_model.pt"

checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_37_32_n10_model.pt"
checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_46_48_n15_model.pt"
checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_56_00_n20_model.pt"


checkpoints = [baseline, small_model, checkpoint_10, checkpoint_15, checkpoint_20]
for i, fname in enumerate(filenames):

    img = tf.imread(fname)

    print(i, fname) 

    fig, ax = plt.subplots(1, len(checkpoints) + 1, figsize=(15, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")


    titles = ["Baseline", "Update", "Checkpoint 10", "Checkpoint 15", "Checkpoint 20"]
    for j, (checkpoint, title) in enumerate(zip(checkpoints, titles), 1):
        model = SegmentationModel(checkpoint=checkpoint, num_classes=3)

        mask = model.inference(img)

        ax[j].imshow(mask)
        ax[j].set_title(title)

    save_path = "results34"
    os.makedirs(save_path, exist_ok=True)
    fig.savefig(os.path.join(save_path, f"{os.path.basename(fname).replace('tif', 'png')}"))

    plt.show()

### Pattern Testing


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, patterning
import matplotlib.pyplot as plt

protocol_path = "/home/patrick/github/autoliftout/liftout/protocol/protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path)

In [ ]:
patterns_fn = {f"{pattern.name}": pattern for pattern in patterning.__PATTERNS__}

print(patterns_fn)

In [ ]:
my_pattern

In [ ]:
from fibsem.structures import FibsemPatternSettings, FibsemPattern

rect_pattern = patterns_fn["Rectangle"]()

rp = rect_pattern.define(protocol={"pattern": "Rectangle", "width": 10e-6, "height": 5e-6, 
                                              "depth": 2e-6, "rotation": 0, 
                                              "centre_x": 0, "centre_y": 0, 
                                              "scan_direction": "TopToBottom", "cleaning_cross_section": True})


print(rp)

In [ ]:



pattern_fn = patterns_fn["Trench"]()
print(pattern_fn.rk)
# pattern = pattern_fn.define(protocol=settings.protocol["jcut"])

# for p in pattern:
    # print(p)

print("---------------------------------------")

In [ ]:
for p in patterns_fn:
    print(p, patterns_fn[p], patterns_fn[p]())

### Test AutoFOCUS



In [3]:
%load_ext autoreload
%autoreload 2

import sys
from fibsem import utils, calibration, patterning
from fibsem.structures import BeamType

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
microscope, settings = utils.setup_session()



2023-03-10 11:17:05,910 — root — INFO — connect_to_microscope:3117 — Connected to Demo Microscope
2023-03-10 11:17:05,911 — root — INFO — setup_session:227 — Finished setup for session: demo_2023-03-10-11-17-05AM


In [7]:
# default
calibration.auto_focus_beam(microscope, beam_type=BeamType.ELECTRON)

2023-03-10 11:17:06,950 — root — INFO — auto_focus:3151 — Auto focus: BeamType.ELECTRON
2023-03-10 11:17:06,952 — root — INFO — auto_focus:3152 — I'm focusing my laser...


In [13]:
# sharpness
calibration.auto_focus_beam(microscope, settings, beam_type=BeamType.ELECTRON, mode="sharpness", num_steps=10)

2023-03-10 11:21:22,614 — root — INFO — get:3317 — Getting working_distance (BeamType.ELECTRON)
2023-03-10 11:21:22,616 — root — INFO — auto_focus_beam:49 — sharpness (accutance) based auto-focus routine
2023-03-10 11:21:22,617 — root — INFO — auto_focus_beam:50 — initial working distance: 2.95e-03
2023-03-10 11:21:22,618 — root — INFO — auto_focus_beam:62 — image 0: 2.70e-03
2023-03-10 11:21:22,623 — root — INFO — set:3339 — Setting working_distance to 0.0026999999999999993 (BeamType.ELECTRON)
2023-03-10 11:21:22,624 — root — INFO — autocontrast:3148 — Autocontrast: BeamType.ELECTRON
2023-03-10 11:21:22,788 — root — INFO — auto_focus_beam:62 — image 1: 2.75e-03
2023-03-10 11:21:22,789 — root — INFO — set:3339 — Setting working_distance to 0.0027499999999999994 (BeamType.ELECTRON)
2023-03-10 11:21:22,790 — root — INFO — autocontrast:3148 — Autocontrast: BeamType.ELECTRON
2023-03-10 11:21:22,948 — root — INFO — auto_focus_beam:62 — image 2: 2.80e-03
2023-03-10 11:21:22,949 — root — INFO